In [5]:
import pandas as pd
import numpy as np
import datetime as DT
import requests
import json 
import warnings
import csv

# disattiva i warning che escono
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# lista degli indicatori da confrontare
indicators = [
    "mask",
    "worried_catch_covid",
    "belief_masking_effective",
    "received_news_local_health",
    "received_news_experts",
    "received_news_who",
    "received_news_govt_health",
    "received_news_politicians",
    "received_news_journalists",
    "received_news_friends",
    "trust_covid_info_local_health",
    "trust_covid_info_experts",
    "trust_covid_info_who",
    "trust_covid_info_govt_health",
    "trust_covid_info_politicians",
    "trust_covid_info_journalists",
    "trust_covid_info_friends"
]

# Dizionario per salvare i paesi per ogni indicatore
indicator_country_map = {}

# Ciclo su tutti gli indicatori
for i, indicator in enumerate(indicators, start=1):
    #print(f"\nAnalizzando indicatore {i}/{len(indicators)}: {indicator}")
    url = f"https://covidmap.umd.edu/apiv2/resources?indicator={indicator}&type=daily&country=all&region=all&daterange=20210521-20220625"
    response = json.loads(requests.get(url, verify=False).text)
    df = pd.DataFrame.from_dict(response['data'])
    df["survey_date"] = pd.to_datetime(df['survey_date'], errors='coerce')
    df.to_csv(f'Data/all_{indicator}.csv', index=False)

    country_counts = df.groupby('country').size()     # conta le righe per paese
    # Filtra i paesi con più di 300 righe
    countries_over_300 = country_counts[country_counts > 300].index.tolist()
    indicator_country_map[indicator] = countries_over_300

# Stampa risultato finale
print("\n===> Paesi con più di 300 righe per ciascun indicatore:")
for indicator, countries in indicator_country_map.items():
    print(f"{indicator}: {len(countries)} paesi -> {countries}")

# Trova i paesi comuni a tutte le liste
common_countries = set.intersection(*map(set, indicator_country_map.values()))
print("===> Paesi comuni a tutti gli indicatori con >300 righe:")
print(sorted(common_countries))




===> Paesi con più di 300 righe per ciascun indicatore:
mask: 69 paesi -> ['Algeria', 'Argentina', 'Australia', 'Austria', 'Bangladesh', 'Belgium', 'Bolivia', 'Brazil', 'Bulgaria', 'Canada', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Czech Republic', 'Denmark', 'Ecuador', 'Egypt', 'El Salvador', 'Finland', 'France', 'Germany', 'Greece', 'Guatemala', 'Honduras', 'Hong Kong', 'Hungary', 'India', 'Indonesia', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Japan', 'Jordan', 'Kenya', 'Malaysia', 'Mexico', 'Morocco', 'Myanmar', 'Nepal', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto Rico, U.S.', 'Romania', 'Russia', 'Saudi Arabia', 'Serbia', 'Slovakia', 'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'Ukraine', 'United Kingdom', 'Uruguay', 'Venezuela', 'Vietnam']
worried_catch_covid: 51 paesi -> ['Argentina', 'Australia', 'Austria', 'Bangladesh', 'Belgium', 'Brazil', 'Canada', '

In [6]:
da_eliminare=[]
# CREO I CSV CON SOLO I COMMON_COUNTRIES
for indicator in indicators:

    url = f"https://covidmap.umd.edu/apiv2/resources?indicator={indicator}&type=daily&country=all&region=all&daterange=20210521-20220625"
    response = json.loads(requests.get(url, verify=False).text)
    df = pd.DataFrame.from_dict(response['data'])
    df["survey_date"] = pd.to_datetime(df['survey_date'], errors='coerce')
    
    # Filtra i paesi comuni
    df_common = df[df['country'].isin(common_countries)]

    # Salva il DataFrame filtrato in un CSV con solo la colonna 'country'
    df_common[['country']].to_csv(f'Data/common_{indicator}.csv', index=False)
    
    #VERIFICA SE CI SONO BUCHI DI DATI E QUINDI NON SONO OMOGENEI
    for country in common_countries:
        country_df = df_common[df_common['country'] == country]
      
        country_df = country_df.sort_values('survey_date')  # Ordina per data
        
        # calcola la differenza tra le date consecutive
        country_df['date_diff'] = country_df['survey_date'].diff().dt.days
        # verifica se ci sono differenze elevate (indicando un "buco" nei dati)
        gaps = country_df[country_df['date_diff'] > 7]
        if not gaps.empty:
            #print(f"Ci sono buchi nei dati per {country} con l'indicatore {indicator}:")
            #print(gaps[['country', 'survey_date', 'date_diff']])
            da_eliminare.append(country)
da_eliminare = list(set(da_eliminare))
da_eliminare

['South Korea', 'South Africa', 'Bangladesh', 'Israel']

In [ ]:
#elimino le country che sono in da_eliminare perchè hanno dei buchi maggiori di 7 giorni (scelto da me)
for country in da_eliminare:
    while country in common_countries:
        common_countries.remove(country)
        
#metto i common countries su un file csv 
with open("csv/common_countries.csv", mode="w", newline='', encoding="utf-8") as file_csv:
    writer = csv.writer(file_csv)
    writer.writerow(["country"])
    for country in common_countries:
        writer.writerow([country])

In [ ]:
#per ogni paese e data mi salvo tutti i vari indicatori e sample-size e pct. 

# Leggi tutti i file CSV e combinali
dfs = []
for indicator in indicators:
    df = pd.read_csv(f'Data/all_{indicator}.csv')
    df['survey_date'] = pd.to_datetime(df['survey_date'])
    df = df[['country', 'survey_date', 'indicator', 'sample_size', 'pct']]
    
    # Filtra solo i paesi comuni 
    if 'common_countries' in locals():
        df = df[df['country'].isin(common_countries)]
    
    dfs.append(df)

# Combina tutti i dati
all_data = pd.concat(dfs)

# Pivota il DataFrame per ottenere il formato wide
wide_df = all_data.pivot_table(
    index=['country', 'survey_date'],
    columns='indicator',
    values=['sample_size', 'pct'],
    aggfunc='first'  # Prende il primo valore se ci sono duplicati
)

# Appiattisci le colonne multi-indice
wide_df.columns = [f"{col[0]}_{col[1]}" for col in wide_df.columns]

# Resetta l'indice
wide_df = wide_df.reset_index()

# Riordina le colonne 
cols_order = ['country', 'survey_date']
for indicator in indicators:
    cols_order.extend([f'sample_size_{indicator}', f'pct_{indicator}'])

# Seleziona solo le colonne esistenti
cols_order = [col for col in cols_order if col in wide_df.columns]
wide_df = wide_df[cols_order]

# Salva il risultato
wide_df.to_csv('csv/FULL_DATA1.csv', index=False)